In [18]:
import os, sys
import numpy as np
import itertools
from collections import defaultdict

from tqdm.notebook import trange
from tqdm import tqdm

In [19]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

In [22]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

In [20]:
# load functions from another notebook
%run helpers.ipynb

In [21]:
pooling_model = load_model('pooling')

In [111]:
target_item_pos = [3]

res = []
tuples_length = 3
top_k = 10
model = pooling_model
num_users = 50
materialized_pred_scores = defaultdict(list)

with tqdm(total=len(target_item_pos), desc='target position loop') as pbar:
    for pos in target_item_pos:
#         best_tot_loss_data[pos] = []
        pbar.update(10)

        for user_id in trange(1, num_users, desc='users loop', leave=False):
            seq_size = len(test.sequences[test.user_ids == user_id])

            for j in range(seq_size):
                if all(v > 0 for v in test.sequences[test.user_ids == user_id][j]):
                    items_interacted = test.sequences[test.user_ids == user_id][j]

                    predictions = -model.predict(items_interacted)
                    predictions[items_interacted] = StaticVars.FLOAT_MAX

                    target_item = predictions.argsort()[min(top_k, int(pos)) - 1]

                    for r in range(tuples_length):
                        combs = itertools.combinations(items_interacted, r + 1)
                        for c in combs:
                            preds = -model.predict(list(set(items_interacted).difference(set(c))))
                            preds[items_interacted] = StaticVars.FLOAT_MAX

                            materialized_pred_scores[user_id].append([c, preds[target_item], preds[preds.argsort()[top_k - 1]]])
                            
#                     for r in range(2, tuples_length + 2):
#                         combs = itertools.combinations(items_interacted, r)
#                         for c in combs:
#                             preds = -model.predict(list(set(items_interacted).difference(set(c))))
#                             preds[items_interacted] = StaticVars.FLOAT_MAX
                            
                            

target position loop:   0%|          | 0/1 [00:00<?, ?it/s]

target position loop: 10it [00:32,  3.23s/it]              


In [116]:
after_top_k = []
before_top_k = []
for k, val in materialized_pred_scores.items():
    for comb in val:
        if (comb[1] / comb[2]) < 1:
#             print(k, comb)
            before_top_k.append(k)
        else: after_top_k.append(k)